### NX-421: Neural signals and signal processing
# Miniproject 1 - Variant 3

### Camille Dorster, Toufan Kashaev, Johan Bordet

### Dataset Description
The dataset used is a snippet issued from the Human Connectome Project. In particular, this task was adapted from a previously developed one by Buckner and colleagues (Buckner et al. 2011 [2]). Participants are shown visual cues asking them to either tap their left or right fingers, or squeeze their left or right toes, or move their tongue, the goal being to map motor areas. Each block of a movement type lasted ~12 seconds (10 movements), and is preceded by a ~3 second cue. In each of the two runs, there are 13 blocks, with 2 of tongue movements, 4 of hand movements (2 right and 2 left), and 4 of foot movements (2 right and 2 left). In addition, there are 3 15-second fixation blocks per run.

### Importing the relevant libraries

In [1]:
import sys
import os

#####################
# Import of utils.py functions
#####################
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/data"

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri
from nilearn.image import load_img, new_img_like, concat_imgs
from nilearn.masking import compute_epi_mask, apply_mask, unmask

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

# Utility function used to load json from file name
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

In [2]:
%gui wx

Gtk-Message: 22:20:48.017: Failed to load module "canberra-gtk-module"


### Starting FSLeyes

In [3]:
################
# Start FSLeyes
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

22:20:52: Debug: Adding duplicate image handler for 'Windows bitmap file'
22:20:52: Debug: Adding duplicate animation handler for '1' type
22:20:52: Debug: Adding duplicate animation handler for '2' type
22:20:52: Debug: Adding duplicate image handler for 'Windows bitmap file'
22:20:52: Debug: Adding duplicate animation handler for '1' type
22:20:52: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:12663): Gtk-CRITICAL **: 22:20:52.427: gtk_window_resize: assertion 'height > 0' failed


### Preparing the paths and structure of the data in folders

In [4]:
sample_path = "/home/jovyan/data"
dataset_id = 'mydataset'
subject_id = '101410' 
subject_dir = 'sub-{}'.format(subject_id)

bids_root = op.join(sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
preproc_subject_path = op.join(preproc_root, subject_dir)
preproc_anat_path = op.join(preproc_root, subject_dir, 'anat')
preproc_func_path = op.join(preproc_root, subject_dir, 'func')
preproc_fmap_path = op.join(preproc_root, subject_dir, 'fmap')

###################
# Create folders relevant for preprocessing.
###################
mkdir_no_exist(sample_path)
mkdir_no_exist(bids_root)
mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)
mkdir_no_exist(preproc_subject_path)
mkdir_no_exist(preproc_anat_path)
mkdir_no_exist(preproc_func_path)
mkdir_no_exist(preproc_fmap_path)

In [5]:
print_dir_tree(bids_root, max_depth=4)

|mydataset/
|--- derivatives/
|------ preprocessed_data/
|--------- sub-101410/
|------------ anat/
|------------ fmap/
|------------ func/
|--- sub-101410/
|------ anat/
|--------- sub-101410_T1w.nii.gz
|------ func/
|--------- events_LR.csv
|--------- events_RL.csv
|--------- sub-101410_tfMRI_MOTOR_LR.nii
|--------- sub-101410_tfMRI_MOTOR_RL.nii
|--------- task-motor_bold.json
|--------- .ipynb_checkpoints/


## Step 1 : skull stripping

### Applying BET to get the betted brain mask

In [6]:
resulting_mask_path = op.join(preproc_anat_path, 'sub-{}_T1w_mask'.format(subject_id))
anatomical_path = op.join(bids_root, subject_dir, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
betted_brain_path = op.join(preproc_anat_path, 'sub-{}_T1w'.format(subject_id))

os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R')) # Robust parameter used

print("Done with BET.")

Done with BET.


In [7]:
# Display the mask
fsleyesDisplay.load(resulting_mask_path)

### Applying the betted brain mask to the original brain

In [8]:
anatomical_path = op.join(bids_root, subject_dir, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id)) # The original brain
betted_brain_path = op.join(preproc_anat_path, 'sub-{}_T1w.nii.gz'.format(subject_id)) # The brain without skull is in the derivatives folder
resulting_mask_path = op.join(preproc_anat_path, 'sub-{}_T1w_mask'.format(subject_id)) # The mask to use

os.system('fslmaths {} -mas {} {}'.format(anatomical_path, resulting_mask_path, betted_brain_path))

print("Mask applied.")

Mask applied.


In [9]:
# Display the betted brain
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)


(ipykernel_launcher.py:12663): Gdk-WARNING **: 22:21:32.753: gdkdrawable-x11.c:952 drawable is not a pixmap or window


## Step 2 : tissue segmentation

In [10]:
fast_target = betted_brain_path

[os.remove(f) for f in glob.glob(op.join(preproc_anat_path, '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_anat_path, 'sub-101410_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

{}

In [ ]:
# Display the segmented brain
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## Step 3 : Variance normalization and concatenation of the runs

In [ ]:
# Display the betted brain
LR_tfMRI_data_path = op.join(bids_root, 'sub-101410', 'func', 'sub-{}_tfMRI_MOTOR_LR'.format(subject_id))
RL_tfMRI_data_path = op.join(bids_root, 'sub-101410', 'func', 'sub-{}_tfMRI_MOTOR_RL'.format(subject_id))

concat_tfMRI_data_path = op.join(preproc_func_path, 'sub-{}_task-motor_concat_scaled.nii.gz'.format(subject_id))

In [ ]:
# $ python -m pip install fmriprep-docke
run_files = [
    LR_tfMRI_data_path + '.nii',
    RL_tfMRI_data_path + '.nii',
]

# 1) Make a consistent brain mask (intersection across runs for safety)
runs = [load_img(r) for r in run_files]
mask = compute_epi_mask(imgs[0])
for run in runs[1:]:
    mask = new_img_like(mask, (mask.get_fdata()>0) & (compute_epi_mask(run).get_fdata()>0))

scaled_runs = []
for img in imgs:
    # 2) mask to get (T, V)
    X = apply_mask(img, mask)                # shape (T, V)
    # 3) global mean & demean
    mu = X.mean()
    X = X - mu
    # 4) global variance & rescale to unit variance
    # ddof=1 for sample variance; add tiny eps for numerical safety
    var = X.var(ddof=1)
    X = X / np.sqrt(var + 1e-8)
    # 5) put back to 4D image with original affine/header
    scaled_img = unmask(X, mask)
    scaled_runs.append(scaled_img)

# 6) concatenate timewise AFTER scaling
concat_img = concat_imgs(scaled_runs)  # -> sub-101410_task-motor_concat_scaled.nii.gz (in memory)
nib.save(concat_img, concat_tfMRI_data_path)


In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(concat_tfMRI_data_path)

## Step 4 : motion correction

In [ ]:
from fsl.wrappers import mcflirt

path_moco_data = op.join(preproc_func_path, 'sub-{}_tfMRI_MOTOR_LR_moco'.format(subject_id))
mcflirt(infile=concat_tfMRI_data_path,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(path_original_data)
fsleyesDisplay.load(path_moco_data)